In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def snell(theta,n=1.77):
    return np.arcsin(np.sin(theta)/n)  # refraction at air-ice interface


def get_depth(r0,h,theta,n=1.77):
    theta_ice = snell(theta,n)
        
    r_air, x_air = h/np.cos(theta), h*np.tan(theta)
    r_ice = r0 - r_air
    d, x_ice = r_ice*np.cos(theta_ice), r_ice*np.sin(theta_ice)

    return d, x_air+x_ice


def get_theta(x,h,d,n=1.77):
    # TODO: this is an approximation
    theta =  (x)/(h+d/n)
    return theta


def get_range(x,h,theta,n=1.77):
    theta_ice = snell(theta,n)
    
    r_air, x_air = h/np.cos(theta), h*np.tan(theta)
    x_ice = x - x_air
    r_ice = x_ice/np.sin(theta_ice)

    return r_air+r_ice

In [ ]:
# measurements
r0 = 1000 # range
h = 200 # height above ice surface
dx = .1
Xs = np.arange(-100.,100+dx,dx) # along-track distances within the aperture

# Initialize the figure
plt.figure(figsize=(4,3))
plt.axhline(0,color='grey',ls=':')
plt.axvline(0,color='grey',ls=':')

# for a given squint angle (theta) find the depth in ice 
# and along-track distance (x) from center of aperture to target
theta = 1e-10*np.pi/180.
d, x = get_depth(r0,h,theta)

# range offset within aperture - air only so simple geometry
R1_air = r0 - np.sqrt(r0**2.+(Xs-x)**2.)
plt.plot(Xs,R1_air,'k',ls='--',label='Nadir Air Only')

# range offset within aperture - with ice so ray bending
R1_ice = np.empty_like(Xs)
for i,x_eps in enumerate(Xs):
    theta = get_theta(x-x_eps,h,d)
    R1_ice[i] = r0 - get_range(x-x_eps,h,theta)
plt.plot(Xs,R1_ice,'k',label='Nadir Air/Ice')

# for a given squint angle (theta) find the depth in ice 
# and along-track distance (x) from center of aperture to target
theta = -3.*np.pi/180.
d, x = get_depth(r0,h,theta)

# range offset within aperture - air only so simple geometry
R2_air = h + d - np.sqrt((h+d)**2.+(Xs-x)**2.)
plt.plot(Xs,R2_air,'indianred',ls='--',label='Squinted Air Only')

# range offset within aperture - with ice so ray bending
R2_ice = np.empty_like(Xs)
for i,x_eps in enumerate(Xs):
    theta = get_theta(x-x_eps,h,d)
    R2_ice[i] = h + d - get_range(x-x_eps,h,theta)
plt.plot(Xs,R2_ice,'indianred',label='Squinted Air/Ice')

plt.legend(fontsize=8)

plt.ylim(-5,1)
plt.xlim(min(Xs),max(Xs))
plt.xlabel('Along-track distance (m)')
plt.ylabel('Range Offset (m)')

plt.gca().yaxis.tick_right()
plt.gca().yaxis.set_label_position("right")

plt.tight_layout()
plt.savefig('Range_Offsets.pdf')

In [ ]:
def get_matched_filter(r,fc=150e6,c=3e8):
    # expected range and phase along track
    phi = 4.*np.pi*fc*r/c
    # to be used for matched filter
    C = np.exp(-1j*phi)

    return phi,C
    
plt.figure(figsize=(4,3))

ax1=plt.subplot(211)
plt.tick_params(labelbottom=False)
plt.xlim(min(Xs),max(Xs))
phi,C = get_matched_filter(R1_air)
plt.plot(Xs,C,'k--')
phi,C = get_matched_filter(R1_ice)
plt.plot(Xs,C,'k-')
plt.text(0.03,0.8,'A',transform=ax1.transAxes,weight='bold',
        bbox=dict(facecolor='w', edgecolor='k',pad=2,lw=2))

ax2 = plt.subplot(212)
plt.xlim(min(Xs),max(Xs))
phi,C = get_matched_filter(R2_air)
plt.plot(Xs,C,'--',c='maroon')
phi,C = get_matched_filter(R2_ice)
plt.plot(Xs,C,'-',c='maroon')
plt.xlabel('Along-track distance (m)')
plt.ylabel('Ref. Waveform')
plt.text(0.03,0.8,'B',transform=ax2.transAxes,weight='bold',
        bbox=dict(facecolor='w', edgecolor='k',pad=2,lw=2))

plt.tight_layout()
plt.savefig('Reference_Waveforms.pdf')

In [ ]:
def get_depth(r0,h,theta,n=1.77):
    theta_ice = snell(theta,n)
        
    r_air, x_air = h/np.cos(theta), h*np.tan(theta)
    r_ice = r0 - r_air
    d, x_ice = r_ice*np.cos(theta_ice), r_ice*np.sin(theta_ice)

    return d, x_air, x_ice

def get_depth_x(x,h,theta,n=1.77):
    # Snells law
    theta_ice = np.arcsin(np.sin(theta)/n)  # refraction at air-ice interface
    
    r_air, x_air = h/np.cos(theta), h*np.tan(theta)
    x_ice = x - x_air
    d, r_ice = x_ice/np.tan(theta_ice), x_ice/np.sin(theta_ice)

    return d, x_air, x_ice
    
# measurements
r0 = 1000 # range
h = 200 # height above ice surface

theta = -3.*np.pi/180.
d0, x_air, x_ice = get_depth(r0,h,theta)
x = x_air + x_ice

plt.figure()
plt.axis('off')
plt.axhline(-h,color='lightblue',zorder=2)
plt.plot(0,-h-d0,'ko',zorder=2)
plt.plot([-x+x_air,-x+x_air],[0,-2.*h],c='grey',ls=':')

plt.plot([-x,-x+x_air],[0,-h],'-',c='grey',zorder=1)
plt.plot([-x+x_air,-x+x_air+x_ice],[-h,-(h+d0)],'-',c='grey',zorder=1)

dx = 2
for x_eps in np.arange(-20,20+dx,dx):
    theta = get_theta(x-x_eps,h,d0)
    d, x_air, x_ice = get_depth_x(x-x_eps,h,theta)
    plt.plot([-x+x_eps,-x+x_air+x_eps],[0,-h],'-',c='grey',zorder=-1,alpha=0.25)
    plt.plot([-x+x_air+x_eps,-x+x_air+x_ice+x_eps],[-h,-(h+d)],'-',c='grey',zorder=-1,alpha=0.25)

plt.savefig('SAR_geometry.pdf')